# Model Building
Build a basic LSTM to predict for calendar year 2024

In [1]:
from _Setup import *

Installing dependencies from requirements.txt...
All dependencies installed successfully.


In [2]:
sector_train = pd.read_csv(sector_train_dropped_csv_path)


In [3]:
sector_train.head()

,Component,Land Border Region,Area of Responsibility,Demographic,Citizenship,Title of Authority,Encounter Type,Encounter Count,Year-Date
0,Office of Field Operations,Northern Land Border,Boston Field Office,FMUA,BRAZIL,Title 8,Inadmissibles,2,2019-10-01
1,Office of Field Operations,Northern Land Border,Boston Field Office,FMUA,OTHER,Title 8,Inadmissibles,29,2019-10-01
2,Office of Field Operations,Northern Land Border,Boston Field Office,Single Adults,BRAZIL,Title 8,Inadmissibles,1,2019-10-01
3,Office of Field Operations,Northern Land Border,Boston Field Office,Single Adults,CANADA,Title 8,Inadmissibles,1031,2019-10-01
4,Office of Field Operations,Northern Land Border,Boston Field Office,Single Adults,"CHINA, PEOPLES REPUBLIC OF",Title 8,Inadmissibles,9,2019-10-01
